In [1]:
import sys
import os

# Insert the parent directory of "app" into sys.path
# so that Python recognizes "app" as an importable package.
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(parent_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv()  # This looks for .env in the current working directory

True

In [3]:
"""
import time
start = time.time()
from app.workflows.naive_text2cypher import NaiveText2CypherFlow
end = time.time()
print(f"Import took {end - start:.2f} seconds")
flow = NaiveText2CypherFlow(verbose=True, timeout=30)
"""

'\nimport time\nstart = time.time()\nfrom app.workflows.naive_text2cypher import NaiveText2CypherFlow\nend = time.time()\nprint(f"Import took {end - start:.2f} seconds")\nflow = NaiveText2CypherFlow(verbose=True, timeout=30)\n'

In [4]:
import time
start = time.time()
from app.workflows.naive_text2cypher_retry import NaiveText2CypherRetryFlow
end = time.time()
print(f"Import took {end - start:.2f} seconds")
flow = NaiveText2CypherRetryFlow(verbose=False, timeout=30)

/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Import took 10.63 seconds


In [5]:
import pandas as pd

test_df = pd.read_csv('test_data.csv')
test_df.head()

,Question
0,Who acted in Tom Hanks’s highest-rated movie?
1,Which movie starring Meg Ryan has the most act...
2,Who directed the most recent movie starring Ha...
3,Which actors starred together in the same movi...
4,What is the highest-rated movie from the 1990s...


In [ ]:
results = []
latencies = []
for question in test_df['Question']:
    print(question)
    start = time.time()
    try:
        data = await flow.run(input=question)
    except:
        data = "timeout"
    end = time.time()
    latencies.append(end - start)
    results.append(data)
    print(data)
    

Who acted in Tom Hanks’s highest-rated movie?
{'cypher': "MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie)\nWITH m\nORDER BY m.imdbRating DESC LIMIT 1\nMATCH (m)<-[:ACTED_IN]-(a:Person)\nRETURN a.name, m.title", 'question': 'Who acted in Tom Hanks’s highest-rated movie?', 'answer': "Based on the context provided, Tom Hanks's highest-rated movie is *From the Earth to the Moon*. The actors who acted in this movie alongside Tom Hanks are:\n\n- David Andrews\n- Lane Smith\n- Nick Searcy\n\nThis conclusion is drawn from the Cypher query and the returned data, which lists the actors (`a.name`) and the movie title (`m.title`)."}
Which movie starring Meg Ryan has the most actors in common with a Tom Hanks movie?
{'cypher': "MATCH (meg:Person {name: 'Meg Ryan'})-[:ACTED_IN]->(megMovie:Movie)<-[:ACTED_IN]-(megActors:Person),\n      (tom:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(tomMovie:Movie)<-[:ACTED_IN]-(tomActors:Person)\nWHERE megActors = tomActors\nWITH megMovie, COUNT(DISTINCT 

Exception in callback Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-f3f5e4b803f8', bound_args=<BoundArgumen...is before?'})>, instance=<app.workflow...t 0x34ebc8850>, context=<_contextvars...t 0x34eeee3c0>)(<WorkflowHand... 30 seconds')>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273
handle: <Handle Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-f3f5e4b803f8', bound_args=<BoundArgumen...is before?'})>, instance=<app.workflow...t 0x34ebc8850>, context=<_contextvars...t 0x34eeee3c0>)(<WorkflowHand... 30 seconds')>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273>
Traceback (most recent call last):
  File "/Users/tomazbratanic/anaconda3/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/Users/tomazbratanic/anaconda3

timeout
What is the highest-rated movie from the 1990s, who directed it, and which other films did that director make?
{'cypher': 'MATCH (m:Movie)-[:DIRECTED]-(d:Person) \nWHERE m.year >= 1990 AND m.year < 2000 \nWITH m, d \nORDER BY m.imdbRating DESC \nLIMIT 1 \nMATCH (d)-[:DIRECTED]->(other:Movie) \nRETURN m.title AS highestRatedMovie, m.imdbRating AS rating, d.name AS director, COLLECT(other.title) AS otherMovies;', 'question': 'What is the highest-rated movie from the 1990s, who directed it, and which other films did that director make?', 'answer': 'The highest-rated movie from the 1990s is **"The Shawshank Redemption"**, with an IMDb rating of **9.3**. It was directed by **Frank Darabont**. Other films directed by Frank Darabont include **"The Majestic"**, **"The Green Mile"**, and **"The Mist"**. \n\nThis information is directly derived from the provided context.'}
For all movies starring Keanu Reeves, find the director who worked the most times with him and list the co-stars acr